**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [5]:
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] # Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [6]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,55832858,None,<p>There may be privacy description <em>(for S...,None,None,0,None,2019-04-24 14:45:59.090000+00:00,None,2019-04-24 14:45:59.090000+00:00,None,None,None,None,7266317,6004032,2,0,None,None
1,55832872,None,<p>How do I set the name of the button to the ...,None,None,3,None,2019-04-24 14:46:52.690000+00:00,None,2019-04-24 14:46:52.690000+00:00,None,None,None,None,5873109,55832746,2,0,None,None
2,55832876,None,<pre><code>go test -v -timeout 30s &lt;path_to...,None,None,0,None,2019-04-24 14:47:09.227000+00:00,None,2019-04-24 14:47:09.227000+00:00,None,None,None,None,5713047,16935965,2,0,None,None
3,55832878,None,<p>I suggest you create one custom pin represe...,None,None,0,None,2019-04-24 14:47:10.297000+00:00,None,2019-04-24 14:47:10.297000+00:00,None,None,None,None,6001090,33622927,2,0,None,None
4,55832879,None,"<p>You have to use <a href=""https://en.cpprefe...",None,None,2,None,2019-04-24 14:47:16.853000+00:00,None,2019-04-24 14:47:16.853000+00:00,None,None,None,None,10765031,55831782,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [8]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,29153531,Finding two points' center of mass on a sphere...,"<p>Please, help me to solve subproblem in my p...",29159108.0,1,0,None,2015-03-19 19:39:56.413000+00:00,NaN,2015-03-20 19:50:00.047000+00:00,2015-03-20 19:50:00.047000+00:00,None,4690388.0,None,4690388,None,1,0,<geometry><k-means>,256
1,29155782,Swift and SpriteKit: Move two sprites as one c...,<p>Is it possible to move two sprites in Swift...,29155959.0,1,0,None,2015-03-19 21:59:37.103000+00:00,NaN,2015-03-20 17:26:02.987000+00:00,NaT,None,NaN,None,144088,None,1,3,<swift><sprite-kit>,256
2,29169095,Android update row int ormlite,<p>I use <code>ormlite</code> in my Android pr...,NaN,1,0,None,2015-03-20 14:37:27.240000+00:00,NaN,2015-03-21 17:28:36.820000+00:00,2015-03-21 17:28:36.820000+00:00,None,179850.0,None,4611080,None,1,0,<java><android><sql-update><ormlite>,256
3,29169869,Google App Engine - DatastoreFailureException:...,<p>My Application on Google App Engine has bee...,NaN,1,0,None,2015-03-20 15:13:28.160000+00:00,2.0,2015-04-21 05:36:08.763000+00:00,NaT,None,NaN,None,3830089,None,1,3,<google-app-engine>,256
4,29182693,Elastic Search Unique Constraint,<p>I am trying to make a function using elasti...,NaN,1,0,None,2015-03-21 12:27:39.820000+00:00,NaN,2015-03-21 12:39:01.760000+00:00,NaT,None,NaN,None,4696897,None,1,1,<node.js><elasticsearch>,256


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [10]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%' 
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config)  # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results =  questions_query_job.to_dataframe()# Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  19190298                       BigQuery Maximum name length      1706645.0
1  19286517  How to load nested data to Google BigQuery usi...      2500544.0
2  19662223  bigquery how to get the jobreference of an ins...      2739973.0
3  19488515                     BigQuery Data Location Setting      2881671.0
4  19497655  Bigquery cancel or stop a batch query job whic...      2739973.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [13]:
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  49940200  <p>I am not really familiar with Apache Zeppel...      4482491.0
1  49941328  <p>BigQuery locations are set on a dataset lev...      5264794.0
2  49985713  <p>If you have the Legacy SQL option, to answe...       266531.0
3  49990127  <p>In this <a href="https://cloud.google.com/b...      9687748.0
4  49991103  <pre><code>bq mk --table --expiration [INTEGER...      9687748.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [15]:
bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

      user_id  number_of_answers
0   4465586.0                  1
1  10862360.0                  2
2  10700562.0                  1
3   6347358.0                  1
4   5705066.0                 10


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  


In [17]:
def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%' + tag + '%'
               GROUP BY a.owner_user_id
               """

    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results


---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*